## 9. Targeted Mutation Analysis

This section compares the results of runs with the `Random` and `Targeted` mutation strategies.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plot style
sns.set_theme(style="whitegrid")

def load_harness_data(strategy_name):
    """Load fitness data from the harness state for a given strategy."""
    base_path = Path(f"target/targeted_mutation_analysis/{strategy_name}")
    harness_path = base_path / "harness_state.json"
    
    if not harness_path.exists():
        print(f"Harness state not found for {strategy_name} at {harness_path}")
        return pd.DataFrame()

    with open(harness_path, 'r') as f:
        harness_state = json.load(f)
    
    outcomes = harness_state.get('archive', [])
    
    data = []
    for outcome in outcomes:
        candidate = outcome.get('candidate', { })
        data.append({
            'generation': candidate.get('generation'),
            'fitness_score': outcome.get('fitness_score'),
            'strategy': strategy_name
        })
    return pd.DataFrame(data)

# Load data for both strategies
df_random = load_harness_data('random')
df_targeted = load_harness_data('targeted')

# Combine into a single DataFrame
df_combined = pd.concat([df_random, df_targeted]).dropna(subset=['generation', 'fitness_score'])
if not df_combined.empty:
    df_combined['generation'] = df_combined['generation'].astype(int)

# ---
# Data processing and aggregation
# ---
if not df_combined.empty:
    # Calculate average fitness per generation
    avg_fitness = df_combined.groupby(['generation', 'strategy'])['fitness_score'].mean().reset_index()

    # Calculate maximum fitness per generation
    max_fitness = df_combined.groupby(['generation', 'strategy'])['fitness_score'].max().reset_index()

    # ---
    # Plotting
    # ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
    fig.suptitle('Comparison of Random vs. Targeted Mutation Strategies', fontsize=16)

    # 1. Average Fitness per Generation
    sns.lineplot(data=avg_fitness, x='generation', y='fitness_score', hue='strategy', ax=axes[0], marker='o')
    axes[0].set_title('Average Fitness per Generation')
    axes[0].set_ylabel('Average Fitness Score')
    axes[0].set_xlabel('')

    # 2. Maximum Fitness per Generation
    sns.lineplot(data=max_fitness, x='generation', y='fitness_score', hue='strategy', ax=axes[1], marker='o')
    axes[1].set_title('Maximum Fitness per Generation')
    axes[1].set_ylabel('Maximum Fitness Score')
    axes[1].set_xlabel('')

    # 3. Distribution of Fitness Scores per Generation
    sns.boxplot(data=df_combined, x='generation', y='fitness_score', hue='strategy', ax=axes[2])
    axes[2].set_title('Distribution of Fitness Scores per Generation')
    axes[2].set_ylabel('Fitness Score')
    axes[2].set_xlabel('Generation')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()
else:
    print("No data loaded. Cannot generate plots.")

### Analysis of Results

The plots above visualize the performance difference between the `Random` and `Targeted` mutation strategies over 5 generations.

1.  **Average Fitness per Generation**: This plot shows the mean fitness of all candidates in each generation. A steeper upward slope indicates a more efficient exploration of the problem space, leading to better solutions on average.
2.  **Maximum Fitness per Generation**: This plot tracks the fitness of the best-performing candidate in each generation. It helps us understand if a strategy is capable of finding high-impact, 'elite' candidates.
3.  **Distribution of Fitness Scores per Generation**: The boxplot shows the spread of fitness scores. A wider spread might indicate greater diversity in the population, while a distribution that shifts upwards over time shows consistent improvement.

By observing these plots, we can determine if the refined `Targeted` mutation strategy is more effective at evolving high-fitness attack candidates compared to the baseline `Random` strategy.